# 05 - Création du dataset des données nettoyées de jobs

Ce notebook génère 1 csv :

- raw_jobs_dataset.csv

Etapes :

- Suppression des lignes avec les valeurs timestamp_start manquantes

- Suppression des colonnes ayant des valeurs uniques

- Remplacement des données maquantes

- Conversion des types des séries en fonction des types des valeurs

# A. Imports

## a) Librairies

In [1]:
import os, math
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

## b) Données

In [2]:
# chemin vers fichier source
filename = 'jobs.csv'
path = '../data/raw/'
# target path to save merge raw jobs dataset
save_csv = '../data/jobs/raw_jobs_dataset.csv'

In [3]:
# # téléchargement dans le repertoire 'data' d'un fichiers 'csv' depuis le blob
# from azure_blob import download_blob_file
# download_blob_file(file_name=filename, local_path=path)

# B. Dataframe

## a) Création

In [4]:
# création d'un dataframe à partir du csv de données
jobs_df = pd.read_csv(os.path.join(path, filename))
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47073 entries, 0 to 47072
Data columns (total 26 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   id                                47073 non-null  int64  
 1   thumbnail                         0 non-null      float64
 2   total_copies                      47027 non-null  float64
 3   started_at                        47073 non-null  object 
 4   ended_at                          47027 non-null  object 
 5   machineId                         47073 non-null  int64  
 6   speed                             47073 non-null  int64  
 7   operator                          47073 non-null  object 
 8   operator_level                    47073 non-null  object 
 9   first_page_image_path_on_machine  47044 non-null  object 
 10  paperHeight                       47073 non-null  int64  
 11  paperWidth                        47073 non-null  int64  
 12  pape

C:\Users\Allan\AppData\Local\Temp\ipykernel_13092\774032103.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  jobs_df = pd.read_csv(os.path.join(path, filename))


In [5]:
jobs_df.head()

,id,thumbnail,total_copies,started_at,ended_at,machineId,speed,operator,operator_level,first_page_image_path_on_machine,paperHeight,paperWidth,paperName,paperThickness,id_on_machine,total_copies_requested,job_thumbnail_id,uses_ifoil,uses_iper,scanner_mode,iper_bvar_count,varnishConsumptionVarnish_3d,varnishConsumptionVarnish_2d,run,total_run,copies_per_run
0,28291,NaN,40.0,2022-02-22 09:43:04.487000,2022-02-22 09:46:07.946000,17,313,User,Operator,D:/IMAGES/Standard/1502734#1/0000001.tif,470,330,NaN,0,1645522984,40,25463.0,True,True,3,2,4.414782,NaN,NaN,NaN,NaN
1,28294,NaN,18.0,2022-02-22 09:47:20.673000,2022-02-22 09:48:57.474000,17,313,User,Operator,D:/IMAGES/Standard/1496447#1/0000001.tif,470,330,NaN,0,1645523240,29,25466.0,True,True,3,2,3.004043,NaN,NaN,NaN,NaN
2,28297,NaN,15.0,2022-02-22 09:50:14.555000,2022-02-22 09:51:50.551000,17,313,User,Operator,D:/IMAGES/Standard/1496447#1/0000001.tif,470,330,NaN,0,1645523414,15,25469.0,True,True,3,2,2.503964,NaN,NaN,NaN,NaN
3,28298,NaN,17.0,2022-02-22 09:52:53.450000,2022-02-22 09:54:26.428000,17,313,User,Operator,D:/IMAGES/Standard/1498393#1/0000001.tif,470,330,NaN,0,1645523573,44,25470.0,True,True,3,2,1.564754,NaN,NaN,NaN,NaN
4,28303,NaN,31.0,2022-02-22 09:56:21.120000,2022-02-22 09:58:44.819000,17,313,User,Operator,D:/IMAGES/Standard/1498393#1/0000001.tif,470,330,NaN,0,1645523780,31,25475.0,True,True,3,2,2.967451,NaN,NaN,NaN,NaN


## b) Nettoyage

### 1. Suppression de lignes

In [6]:
# on cherche les index des lignes sans timestamp de départ
no_start_row = list(jobs_df[jobs_df['started_at'].isna()].index)
no_start_row

[]

### 2. Suppression de colonnes

In [7]:
# suppression des colonnes non pertinentes
col_to_drop = [
    'id',
    'thumbnail',
    'machineId',
    'first_page_image_path_on_machine',
    'job_thumbnail_id'
    ]
jobs_df = jobs_df.drop(col_to_drop, axis=1)

In [8]:
# suppression des colonnes ne contenant que des valeurs nulles
print(jobs_df.columns[jobs_df.isnull().all()].tolist())
jobs_df.dropna(axis=1, how='all', inplace=True)

['varnishConsumptionVarnish_2d']


### 3. Données manquantes

In [9]:
# on liste les colonnes qui contiennent des valeurs nulles
col_with_nan = []
for col in jobs_df.columns:
    if jobs_df[col].isnull().any():
        col_with_nan.append(col)
        print(f'{col} : {jobs_df[col].unique()}') if jobs_df[col].nunique() <= 10 else print(f'{col} : {jobs_df[col].nunique()} - dtype : {jobs_df[col].dtype}')     

total_copies : 219 - dtype : float64
ended_at : 47027 - dtype : object
paperName : [nan 'calque' 'B2']
varnishConsumptionVarnish_3d : 45119 - dtype : float64
run : [nan  1.]
total_run : [nan  1.]
copies_per_run : 107 - dtype : float64


In [10]:
# on remplace les valeurs de 'total_copies' nulles par 0
jobs_df['total_copies'] = jobs_df['total_copies'].fillna(float(0))

In [11]:
# on remplace les datetime de fin maquant par les datetime de début
jobs_df['ended_at'] = jobs_df['ended_at'].fillna(jobs_df['started_at'])

In [12]:
# on remplace les 'paperName' non défini par UNDEFINED
jobs_df['paperName'] = jobs_df['paperName'].fillna('UNDEFINED')

In [13]:
# on remplace les valeurs de 'scanner_mode' nulles par 0
jobs_df['scanner_mode'] = jobs_df['scanner_mode'].fillna(0)

In [14]:
# on remplace les valeurs de 'iper_bvar_count' nulles par 0
jobs_df['iper_bvar_count'] = jobs_df['iper_bvar_count'].fillna(0)

In [15]:
# on remplace les valeurs de 'total_copies' nulles par 0
jobs_df['varnishConsumptionVarnish_3d'] = jobs_df['varnishConsumptionVarnish_3d'].fillna(float(0))

### 4. Conversion des types

In [16]:
jobs_df.dtypes.unique()

array([dtype('float64'), dtype('O'), dtype('int64'), dtype('bool')],
      dtype=object)

#### Object

In [17]:
# on liste les valeurs (si pas plus de 5) des colonnes de type object
object_columns = list(jobs_df.select_dtypes(include=['object']).columns)
for col in object_columns :
    print(f"{col} : {jobs_df[col].unique()}") if jobs_df[col].nunique() <= 5  else print(f"{col} : {jobs_df[col].nunique()} values")

started_at : 47073 values
ended_at : 47073 values
operator : ['User' 'Distributor']
operator_level : ['Operator' 'Distributor']
paperName : ['UNDEFINED' 'calque' 'B2']


In [18]:
# on peut convertir les colonnes contenant des valeurs de temsps au format datetime
jobs_df['started_at'] = pd.to_datetime(jobs_df['started_at'], utc=True)
jobs_df['ended_at'] = pd.to_datetime(jobs_df['ended_at'], utc=True)

#### Float64

In [19]:
float_columns = list(jobs_df.select_dtypes(include=['float64']).columns)
# on liste les valeurs (si pas plus de 5) des colonnes de type object
for col in float_columns:
    print(f"{col} : {jobs_df[col].unique()}") if jobs_df[col].nunique() <= 5  else print(f"{col} : {jobs_df[col].nunique()} values")

total_copies : 219 values
varnishConsumptionVarnish_3d : 45119 values
run : [nan  1.]
total_run : [nan  1.]
copies_per_run : 107 values


In [20]:
# conversion des colonnes de type 'float' dont toutes les valeurs ont des décimales à 0 en type 'int'
for col in float_columns :
# remplacement de nan par 0
    jobs_df[col] = jobs_df[col].fillna(0)
    if not True in (math.modf(value)[0] != float(0) for value in list(jobs_df[col].unique())):
        jobs_df[col] = jobs_df[col].astype('int64')

In [21]:
jobs_df.head(5)

,total_copies,started_at,ended_at,speed,operator,operator_level,paperHeight,paperWidth,paperName,paperThickness,id_on_machine,total_copies_requested,uses_ifoil,uses_iper,scanner_mode,iper_bvar_count,varnishConsumptionVarnish_3d,run,total_run,copies_per_run
0,40,2022-02-22 09:43:04.487000+00:00,2022-02-22 09:46:07.946000+00:00,313,User,Operator,470,330,UNDEFINED,0,1645522984,40,True,True,3,2,4.414782,0,0,0
1,18,2022-02-22 09:47:20.673000+00:00,2022-02-22 09:48:57.474000+00:00,313,User,Operator,470,330,UNDEFINED,0,1645523240,29,True,True,3,2,3.004043,0,0,0
2,15,2022-02-22 09:50:14.555000+00:00,2022-02-22 09:51:50.551000+00:00,313,User,Operator,470,330,UNDEFINED,0,1645523414,15,True,True,3,2,2.503964,0,0,0
3,17,2022-02-22 09:52:53.450000+00:00,2022-02-22 09:54:26.428000+00:00,313,User,Operator,470,330,UNDEFINED,0,1645523573,44,True,True,3,2,1.564754,0,0,0
4,31,2022-02-22 09:56:21.120000+00:00,2022-02-22 09:58:44.819000+00:00,313,User,Operator,470,330,UNDEFINED,0,1645523780,31,True,True,3,2,2.967451,0,0,0


In [22]:
jobs_df.columns

Index(['total_copies', 'started_at', 'ended_at', 'speed', 'operator',
       'operator_level', 'paperHeight', 'paperWidth', 'paperName',
       'paperThickness', 'id_on_machine', 'total_copies_requested',
       'uses_ifoil', 'uses_iper', 'scanner_mode', 'iper_bvar_count',
       'varnishConsumptionVarnish_3d', 'run', 'total_run', 'copies_per_run'],
      dtype='object')

### 5. Outout csv

In [23]:
# sauvegarde du dataset en csv
jobs_df.to_csv(save_csv)